In [ ]:
DEBUG = True


In [ ]:
import subprocess
from ast import literal_eval

def run(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    out, err = process.communicate()
    print(out.decode('utf-8').strip())

print('# CPU')
run('cat /proc/cpuinfo | egrep -m 1 "^model name"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu MHz"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu cores"')

print('# RAM')
run('cat /proc/meminfo | egrep "^MemTotal"')

print('# GPU')
run('lspci | grep VGA')

print('# OS')
run('uname -a')

In [ ]:
!pip install ../input/sacremoses > /dev/null

import sys
sys.path.insert(0, "../input/transformers/")

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
# import tensorflow_hub as hub
import tensorflow as tf
# import bert_tokenization as tokenization
import tensorflow.keras.backend as K
import os
from scipy.stats import spearmanr
from math import floor, ceil
from transformers import *

import seaborn as sns
import string
import re    #for regex

np.set_printoptions(suppress=True)
print(tf.__version__)

#### 1. Read data and tokenizer

Read tokenizer and data, as well as defining the maximum sequence length that will be used for the input to Bert (maximum is usually 512 tokens)

In [ ]:
HAS_ANS = False

df = pd.read_csv('../input/60k-stack-overflow-questions-with-quality-rate/train.csv')
df = pd.read_csv('../input/phd-three-level-feature-extraction/all_features.csv')
df['Y0'] = df['Yi'].apply(lambda x: int(x==0))
df['Y1'] = df['Yi'].apply(lambda x: int(x==1))
df['Y2'] = df['Yi'].apply(lambda x: int(x==2))
del df['Y'], df['Yi'], df['Unnamed: 0']


df_train = df.head(45000)
df_test = df.tail(15000)

if DEBUG:
#     df_train = df_train.head(30000)
    df_test = df_test.head(1000)

df_sub = df_test.copy()
del df_sub['Y0'], df_sub['Y1'], df_sub['Y2']
print('train shape =', df_train.shape)
print('test shape =', df_test.shape)



In [ ]:
output_categories = list(df_train.columns[61:])
TARGET_COUNT = len(output_categories)

In [ ]:
input_categories = list(df_train.columns[7:61])

## correlation

In [ ]:
train = df_train.copy()

In [ ]:
from scipy.spatial.distance import cdist

def calc_corr(df, x_cols, y_cols):
    arr1 = df[x_cols].T.values
    arr2 = df[y_cols].T.values
    corr_df = pd.DataFrame(1 - cdist(arr2, arr1, metric='correlation'), index=y_cols, columns=x_cols)
    return corr_df

number_feature_cols = input_categories[2:]

corr_df = calc_corr(train, output_categories, number_feature_cols)
fig, ax = plt.subplots(figsize=(2, 6))
sns.heatmap(corr_df, ax=ax)

print(corr_df)

# prep

In [ ]:
from sklearn.model_selection import train_test_split

#Particiona o data set originalmente Train em Train(Treino) e Val(validação)
X_train, X_val, Y_train, Y_val = train_test_split(df_train[input_categories], 
                                                                  df_train[output_categories], 
                                                                  test_size=0.2, 
                                                                  random_state=42)

X_train.shape, Y_train.shape, X_val.shape, Y_val.shape

# 3. models

from https://www.kaggle.com/heyytanay/stack-overflow-qa-classification-87-acc#Modelling

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import random
import warnings

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix, plot_confusion_matrix, plot_precision_recall_curve

In [ ]:
def plot_metric(clf, testX, testY, name):
    """
    Small function to plot ROC-AUC values and confusion matrix
    """
    styles = ['bmh', 'classic', 'fivethirtyeight', 'ggplot']

    plt.style.use(random.choice(styles))
    plot_confusion_matrix(clf, testX, testY)
    plt.title(f"Confusion Matrix [{name}]")

In [ ]:
trainX = X_train.copy()
trainY = Y_train['Y0']
validX = X_val.copy()
validY = Y_val['Y0']

#### models

In [ ]:
# Define and fit the classifier on the data
lr_classifier = LogisticRegression(C=1.)
lr_classifier.fit(trainX, trainY)
# Print the accuracy score of the classifier
print(f"Validation Accuracy of Logsitic Regression Classifier is: {(lr_classifier.score(validX, validY))*100:.2f}%")

plot_metric(lr_classifier, validX, validY, "Logistic Regression")

In [ ]:
# Define and fit the classifier on the data
nb_classifier = MultinomialNB()
nb_classifier.fit(trainX, trainY)
MultinomialNB()
# Print the accuracy score of the classifier
print(f"Validation Accuracy of Naive Bayes Classifier is: {(nb_classifier.score(validX, validY))*100:.2f}%")
# Also plot the metric
plot_metric(nb_classifier, validX, validY, "Naive Bayes")